In this notebook we show how to download and process the GHI measurements from [1] into HDF5 files.

```
[1] Sengupta, M.; Andreas, A. (2010). Oahu Solar Measurement Grid (1-Year Archive):
1-Second Solar Irradiance; Oahu, Hawaii (Data); NREL Report No. DA-5500-56506.
http://dx.doi.org/10.5439/1052451
```

# Libraries

In [1]:
import numpy as np
import pandas as pd
import tables as tb
import os, requests, zipfile, glob, warnings, time

from pvlib.location import Location
from pvlib.irradiance import clearsky_index

pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 30)

# Download full data

In [3]:
def get_data(url, unzip=False, out_fold='../data/raw/'):
    for yyyymm in ['2010{:0>2}'.format(mm) for mm in range(3, 13)] + \
                  ['2011{:0>2}'.format(mm) for mm in range(1, 11)]:
        url_curr = url.format(yyyymm)
        fname = os.path.basename(url_curr)
        file_curr = requests.get(url_curr)
        if file_curr.status_code == 200:
            with open(out_fold + fname, 'wb') as to_file:
                to_file.write(file_curr.content)
            os.mkdir(out_fold + yyyymm)
            if unzip:
                with zipfile.ZipFile(out_fold + fname, 'r') as zipped:
                    zipped.extractall(out_fold + yyyymm)
            print(fname + ' downloaded{}.'.format(' and unzipped' if unzip else ''))
        else:
            print('Failed to download {}, status code is {}'.format(url_curr, file_curr.status_code))

In [4]:
url = 'https://midcdmz.nrel.gov/oahu_archive/rawdata/Oahu_GHI/{}.zip'
get_data(url, unzip=True)

201003.zip downloaded and unzipped.
201004.zip downloaded and unzipped.
201005.zip downloaded and unzipped.
201006.zip downloaded and unzipped.
201007.zip downloaded and unzipped.
201008.zip downloaded and unzipped.
201009.zip downloaded and unzipped.
201010.zip downloaded and unzipped.
201011.zip downloaded and unzipped.
201012.zip downloaded and unzipped.
201101.zip downloaded and unzipped.
201102.zip downloaded and unzipped.
201103.zip downloaded and unzipped.
201104.zip downloaded and unzipped.
201105.zip downloaded and unzipped.
201106.zip downloaded and unzipped.
201107.zip downloaded and unzipped.
201108.zip downloaded and unzipped.
201109.zip downloaded and unzipped.
201110.zip downloaded and unzipped.


Location of the 17 (+1) sensors:

<img src="https://midcdmz.nrel.gov/oahu_archive/map.jpg" alt="M1 - Table field" style="width: 900px;"/>

# Metadata and line check

Notes from https://midcdmz.nrel.gov/apps/html.pl?site=oahugrid;page=instruments#RSR:
  - Data Sample rate for Global Horizontal and Tilt is every 1 second with 1 second outputs (data only collected from 5:00 to 20:00 HST).
  - Data Sample rate for RSR Global Horizontal, Direct Normal and Diffuse Horizontal is every 3 seconds with 3 second outputs (data only collected from 5:00 to 20:00 HST).
  - The CR800 is a table based operating system, data tables of equal output rate are merged together and converted to array ("classic") Campbell format after data collection, also any non-numeric values are converted to -99999. 

It seems like data for all dates starts being recorded at 5am and goes up to 8pm.  
Thus, there should be: 15h * 60m * 60s + 1s = 54001 lines in each file, lets check it:

In [2]:
nlines = 54001

for folder in glob.glob('../data/raw/*'):
    if os.path.isdir(folder):
        for fname in glob.glob(folder + '/*'):
            with open(fname) as file:
                curr_lines = sum(1 for line in open(fname))
                if curr_lines != nlines:
                    print('{} has {} lines.'.format(fname, curr_lines))

# Raw data

The final file will look like this:
```
raw_carray.h5 (file)
|-- years (group)
    |-- months (group)
        |-- days (group)
            |-- sensors: time + radiation (carray)
```

| Field # |	Instrument | Units     |
|---------|------------|-----------|
| 1       |  Seconds   |  ss       |
| 2       |  Year      |  -        |
| 3       |  DOY       |  -        |
| 4       |  HST       |  hhmm     |
| 5       |  DH3       |  W/m$^2$  |
| 6       |  DH4       |  W/m$^2$  |
| 7       |  DH5       |  W/m$^2$  |
| 8       |  DH10      |  W/m$^2$  |
| 9       |  DH11      |  W/m$^2$  |
| 10      |  DH9       |  W/m$^2$  |
| 11      |  DH2       |  W/m$^2$  |
| 12      |  DH1       |  W/m$^2$  |
| 13      |  Tilt DH1  |  W/m$^2$  |
| 14      |  AP6       |  W/m$^2$  |
| 15      |  Tilt AP6  |  W/m$^2$  |
| 16      |  AP1       |  W/m$^2$  |
| 17      |  AP3       |  W/m$^2$  |
| 18      |  AP5       |  W/m$^2$  |
| 19      |  AP4       |  W/m$^2$  |
| 20      |  AP7       |  W/m$^2$  |
| 21      |  DH6       |  W/m$^2$  |
| 22      |  DH7       |  W/m$^2$  |
| 23      |  DH8       |  W/m$^2$  |

In [27]:
col_names = ['Seconds', 'Year', 'DOY', 'HST', 'DH3', 'DH4', 'DH5', 'DH10', 
             'DH11', 'DH9', 'DH2', 'DH1', 'Tilt_DH1', 'AP6', 'Tilt_AP6', 
             'AP1', 'AP3', 'AP5', 'AP4', 'AP7', 'DH6', 'DH7', 'DH8', ]
sensors = ['ap01', 'ap03', 'ap04', 'ap05', 'ap06', 'ap07', 'dh01', 'dh02', 
           'dh03', 'dh04', 'dh05', 'dh06', 'dh07', 'dh08', 'dh09', 'dh10', 'dh11']
map_sensors = dict({
    'ap01': 'AP1',
    'ap03': 'AP3',
    'ap04': 'AP4',
    'ap05': 'AP5',
    'ap06': 'AP6',
    'ap07': 'AP7',
    'dh01': 'DH1',
    'dh02': 'DH2',
    'dh03': 'DH3',
    'dh04': 'DH4',
    'dh05': 'DH5',
    'dh06': 'DH6',
    'dh07': 'DH7',
    'dh08': 'DH8',
    'dh09': 'DH9',
    'dh10': 'DH10',
    'dh11': 'DH11'
})

In [166]:
def transfer_raw(in_path, out_path, max_rows=54001, map_sensors=map_sensors):
    # Parameters for the hdf5 file
    filters = tb.Filters(complib='blosc:lz4hc', complevel=9)
    dtype = tb.Float32Atom()
    data_shape = (0, 2)
    chunkshape=(max_rows, 2)
    tic = time.time()
    # Open file and transfer
    with tb.open_file(out_path, mode='a', filters=filters) as hdf5_file, \
         warnings.catch_warnings() as w:
        warnings.simplefilter('ignore', tb.NaturalNameWarning)
        # For every month
        for folder in glob.glob(in_path):
            if os.path.isdir(folder):
                year = int(os.path.basename(folder)[:4])
                month = int(os.path.basename(folder)[4:])
                # For every day inside that month
                for fname in glob.glob(folder + '/*'):
                    day = int(os.path.splitext(os.path.basename(fname))[0][-2:])
                    df = pd.read_csv(fname, names=col_names)
                    # Compute proper timestamp
                    df.HST = df.HST.apply(lambda x: str(x).rjust(4, '0'))
                    grp_name = '/{:04}/{:02}/{:02}'.format(year, month, day)
                    df['ts'] = df.apply(lambda row: pd.Timestamp(year=year, month=month, 
                                                                 day=day, hour=int(row.HST[0:2]), 
                                                                 minute=int(row.HST[2:]), 
                                                                 second=row.Seconds),
                                        axis=1)
                    df['ts_epoch'] = df.ts.astype('int64')//1e9
                    # Group name
                    grp_name = '/{:04}/{:02}/{:02}'.format(year, month, day)
                    # For every sensor
                    for sensor in sensors:
                        hdf5_file.create_array(obj=df[['ts_epoch', map_sensors[sensor]]].values, 
                                               where=grp_name, name=sensor,
                                               createparents=True) #chunkshape=chunkshape,
                    #return 'It took {} in total'.format(time.strftime('%H:%M:%S', time.gmtime(time.time() - tic)))
    return 'It took {} in total'.format(time.strftime('%H:%M:%S', time.gmtime(time.time() - tic)))

In [167]:
in_path = '../data/raw/*'
out_path = '../data/raw/raw_array.h5'
transfer_raw(in_path, out_path)

'It took 00:26:48 in total'

For reference on how to open and check an `.h5` file:
```
hdf5_file = tb.open_file('../data/raw/test1.h5', mode='r')
hdf5_file.close()
```

# Process the raw data

## Fill gaps

### Distance between sensors 

![](https://delicias.dia.fi.upm.es/nextcloud/index.php/s/fTFqB4Wx6PW8kgJ/preview)

In [2]:
df_st = pd.read_csv('./stations.txt', sep='\t')
df_st

,Station,Latitude,Longitude,Sensor SN,CF [W/m^2/mV]
0,ap01,21.31276,-158.08389,PY66523,110.40
1,ap03,21.31281,-158.08163,PY66524,113.55
2,ap04,21.31141,-158.07947,PY66526,107.58
3,ap05,21.30983,-158.08249,PY66525,116.04
4,ap06,21.30812,-158.07935,PY66521,117.34
5,ap07,21.31478,-158.07785,PY66527,117.61
6,dh01,21.31533,-158.08700,PY66519,107.34
7,dh02,21.31451,-158.08534,PY66505,106.39
8,dh03,21.31236,-158.08463,PY66499,141.76
9,dh04,21.31303,-158.08505,PY66500,118.01


In [5]:
df_st.set_index('Station', inplace=True)

In [6]:
sensors = df_st.index.values
n_sensors = len(sensors)

df_dist = pd.DataFrame(np.empty((n_sensors, n_sensors)).fill(np.nan), 
                       index=sensors, columns=sensors)

for s1 in sensors:
    for s2 in sensors:
        if s1 != s2:
            dist = np.sqrt((df_st.loc[s1, 'Longitude'] - df_st.loc[s2, 'Longitude']) ** 2 +\
                           (df_st.loc[s1, 'Latitude'] - df_st.loc[s2, 'Latitude']) ** 2)
            df_dist.loc[s1, s2] = dist
df_dist

,ap01,ap03,ap04,ap05,ap06,ap07,dh01,dh02,dh03,dh04,dh05,dh06,dh07,dh08,dh09,dh10,dh11
ap01,NaN,0.00226055,0.00462157,0.00324729,0.00649163,0.00636883,0.00403448,0.00227266,0.00084119,0.00119101,0.000882383,0.00304844,0.00328299,0.00374647,0.00299107,0.00189404,0.00273198
ap03,0.00226055,NaN,0.00257402,0.00310161,0.00521483,0.00426255,0.00593189,0.00408094,0.00303356,0.00342707,0.0027184,0.00525004,0.00539679,0.00568465,0.00525161,0.00403094,0.00437961
ap04,0.00462157,0.00257402,NaN,0.00340834,0.00329219,0.00373916,0.00848925,0.00663829,0.00524672,0.0058104,0.00523627,0.00731987,0.00788272,0.00735821,0.00751804,0.00608451,0.00591346
ap05,0.00324729,0.00310161,0.00340834,NaN,0.00357543,0.0067847,0.00711267,0.0054795,0.00331368,0.004098,0.00412918,0.00471653,0.0061589,0.00429042,0.00523399,0.00364726,0.00287127
ap06,0.00649163,0.00521483,0.00329219,0.00357543,NaN,0.00682683,0.0105122,0.00875855,0.00677171,0.00752317,0.0073222,0.00828697,0.00964228,0.00772583,0.0088031,0.00721666,0.00637907
ap07,0.00636883,0.00426255,0.00373916,0.0067847,0.00682683,NaN,0.00916652,0.00749486,0.00719894,0.00740962,0.00650355,0.00941727,0.00901998,0.00994603,0.00927097,0.00823642,0.00863204
dh01,0.00403448,0.00593189,0.00848925,0.00711267,0.0105122,0.00916652,NaN,0.00185149,0.00379971,0.00301538,0.00327341,0.00354683,0.00115974,0.00499626,0.00265272,0.00379231,0.00519597
dh02,0.00227266,0.00408094,0.00663829,0.0054795,0.00875855,0.00749486,0.00185149,NaN,0.0022642,0.00150814,0.00144693,0.00307766,0.00154564,0.00440193,0.00239176,0.00268745,0.0040902
dh03,0.00084119,0.00303356,0.00524672,0.00331368,0.00677171,0.00719894,0.00379971,0.0022642,NaN,0.000790759,0.0012713,0.00222428,0.00287068,0.00292828,0.00227264,0.00105309,0.00205244
dh04,0.00119101,0.00342707,0.0058104,0.004098,0.00752317,0.00740962,0.00301538,0.00150814,0.000790759,NaN,0.000973499,0.0021285,0.002136,0.00318215,0.00186317,0.00129619,0.00262195


In [7]:
dist_map = pd.DataFrame(columns=sensors)
for s in sensors:
    dist_map[s] = df_dist.loc[~df_dist[s].isna(), s].sort_values().index
print(dist_map.shape)
dist_map

(16, 17)


,ap01,ap03,ap04,ap05,ap06,ap07,dh01,dh02,dh03,dh04,dh05,dh06,dh07,dh08,dh09,dh10,dh11
0,dh03,ap01,ap03,dh11,ap04,ap04,dh07,dh05,dh04,dh03,ap01,dh09,dh01,dh06,dh06,dh03,dh10
1,dh05,ap04,ap06,ap03,ap05,ap03,dh02,dh04,ap01,dh05,dh04,dh10,dh09,dh11,dh07,dh06,dh08
2,dh04,dh05,ap05,ap01,ap03,ap01,dh09,dh07,dh10,ap01,dh03,dh08,dh02,dh10,dh10,dh04,dh06
3,dh10,dh03,ap07,dh03,dh11,dh05,dh04,dh01,dh05,dh10,dh02,dh11,dh04,dh09,dh04,dh11,dh03
4,ap03,ap05,ap01,ap04,ap01,ap05,dh05,dh03,dh11,dh02,dh10,dh04,dh06,dh03,dh03,dh09,dh04
5,dh02,dh04,dh05,ap06,dh03,ap06,dh06,ap01,dh06,dh09,dh07,dh03,dh05,dh04,dh08,ap01,ap01
6,dh11,dh10,dh03,dh10,ap07,dh03,dh10,dh09,dh02,dh06,ap03,dh07,dh10,ap01,dh02,dh08,dh09
7,dh09,dh02,dh04,dh04,dh10,dh04,dh03,dh10,dh09,dh07,dh09,ap01,dh03,dh07,dh01,dh05,ap05
8,dh06,ap07,dh11,dh05,dh05,dh02,ap01,dh06,dh07,dh11,dh06,dh02,ap01,dh05,dh11,dh02,dh05
9,ap05,dh11,dh10,dh08,dh04,dh10,dh08,ap03,dh08,dh01,dh01,dh05,dh08,ap05,dh05,dh07,dh07


### Produce new dataset

In [8]:
def fill_gaps(df, date, max_s=30):
    '''
    Need to have the distance map loaded into dist_map.
    '''
    # Copy df to keep track of previous NaNs
    df_new = df.copy()
    # Prepare dict to keep NaN fills
    na_fixed = dict()
    for col in df.columns:
        na_fixed[col] = {'with_prev': 0, 'with_neigh': 0}
    for row in df.index:
        for col in df.columns:
            if df.loc[row, col] == -99999:
                ok = False
                for gap in range(max_s):
                    if row - gap < 0:
                        break
                    if df.loc[row - gap, col] != -99999:
                        df_new.loc[row, col] = df.loc[row - gap, col]
                        ok = True
                        na_fixed[col]['with_prev'] += 1
                        break
                if not ok:
                    for near_s in dist_map.loc[:, col]:
                        if df.loc[row, near_s] != -99999:
                            df_new.loc[row, col] = df.loc[row, near_s]
                            ok = True
                            na_fixed[col]['with_neigh'] += 1
                            break
                if not ok:
                    print('Check out date {}: ({}, {})'.format(date, row, col))
    return df_new.values, na_fixed

In [9]:
def transfer_filling(in_path, out_path, max_rows=54001, n_cols=18):
    # Parameters for the output hdf5 file
    filters = tb.Filters(complib='blosc:lz4hc', complevel=9)
    dtype = tb.Float32Atom()
    data_shape = (0, n_cols)
    chunkshape=(max_rows, n_cols)
    tic = time.time()
    # Open file and transfer
    with tb.open_file(in_path, mode='r') as in_file, \
         tb.open_file(out_path, mode='a', filters=filters) as out_file, \
         warnings.catch_warnings() as w:
        warnings.simplefilter('ignore', tb.NaturalNameWarning)
        # Get sensor names
        sensors = []
        for sensor in in_file.get_node('/2010/03/18'):
            sensors.append(sensor._v_name)
        for year in in_file.get_node('/'):
            for month in in_file.get_node(year):
                for day in in_file.get_node(month):
                    array = np.empty((max_rows, n_cols))
                    for idx, sensor in enumerate(in_file.get_node(day)):
                        array[:, idx + 1] = sensor[:, 1]
                    # Add also timestamps
                    array[:, 0] = sensor[:, 0]
                    # Fill gaps
                    array[:, 1:], na_fixed = fill_gaps(pd.DataFrame(array[:, 1:], columns=sensors), 
                                                       date=day._v_pathname)
                    # Save to new hdf5 file
                    node = out_file.create_carray(obj=array, createparents=True,
                                                  where=month._v_pathname, name=day._v_name)
                    # Add meta: column names and NaN fixed counts
                    node._v_attrs['columns'] = str(['ts', *sensors])
                    node._v_attrs['na_fixed'] = na_fixed
    return 'It took {} in total'.format(time.strftime('%H:%M:%S', time.gmtime(time.time() - tic)))

In [11]:
in_path = '../data/raw/raw_carray.h5'
out_path = '../data/filled_carray.h5'
transfer_filling(in_path, out_path)

'It took 03:36:42 in total'

## Standarize 

### Compute mean and stdev 

In [3]:
out_path = '../data/filled_01s.h5'
day_length, n_days = 54001, 593

with tb.open_file(out_path, mode='r') as out_file:
    sensors = eval(out_file.get_node('/2010/03/18')._v_attrs['columns'])[1:]
    df_stand = pd.DataFrame(index=['mean', 'stdev'], columns=sensors)
    for s_id, sensor in enumerate(sensors):
        sensor_s = np.empty((day_length * n_days,))
        day_c = 0
        for year in out_file.get_node('/'):
            for month in out_file.get_node(year):
                for day in out_file.get_node(month):
                    sensor_s[day_length*day_c:day_length*(day_c+1)] = day[:, s_id + 1]
                    day_c += 1
        df_stand.loc['mean', sensor] = sensor_s.mean()
        df_stand.loc['stdev', sensor] = sensor_s.std()
display(df_stand)

,ap01,ap03,ap04,ap05,ap06,ap07,dh01,dh02,dh03,dh04,dh05,dh06,dh07,dh08,dh09,dh10,dh11
mean,369.583,371.677,370.767,374.43,376.328,375.62,370.251,371.368,377.211,370.59,374.523,375.172,376.577,371.386,375.343,374.901,371.611
stdev,347.86,351.347,348.776,355.133,355.936,357.222,349.585,351.749,363.1,350.484,354.856,354.726,356.311,351.574,363.053,353.144,351.126


In [4]:
df_stand.to_csv('../data/other/mean_sd_stations.csv')

In [5]:
print('For the record, a ts comprising all data for a single sensor (320M of records) takes about {:0.5}MB.'.format(
    sensor_s.nbytes / 10**6))

For the record, a ts comprising all data for a single sensor (320M of records) takes about 256.18MB.


### Produce new dataset

In [108]:
def transfer_stand(in_path, out_path, df_stand, max_rows=54001, n_cols=18):
    # Parameters for the output hdf5 file
    filters = tb.Filters(complib='blosc:lz4hc', complevel=9)
    dtype = tb.Float32Atom()
    data_shape = (0, n_cols)
    chunkshape=(max_rows, n_cols)
    tic = time.time()
    # Open file and transfer
    with tb.open_file(in_path, mode='r') as in_file, \
         tb.open_file(out_path, mode='a', filters=filters) as out_file, \
         warnings.catch_warnings() as w:
        warnings.simplefilter('ignore', tb.NaturalNameWarning)
        # Get sensor names
        sensors = eval(in_file.get_node('/2010/03/18')._v_attrs['columns'])[1:]
        for year in in_file.get_node('/'):
            for month in in_file.get_node(year):
                for day in in_file.get_node(month):
                    # Load data for that day
                    array = day[:, :]
                    # Standarize
                    array[:, 1:] = array[:, 1:] - df_stand.loc['mean', :].values.reshape((1,n_cols-1))
                    array[:, 1:] = array[:, 1:] / df_stand.loc['stdev', :].values.reshape((1,n_cols-1))
                    # Save to new hdf5 file
                    node = out_file.create_carray(obj=array, createparents=True,
                                                  where=month._v_pathname, name=day._v_name)
                    # Add meta: column names and NaN fixed counts
                    node._v_attrs['columns'] = day._v_attrs['columns']
                    node._v_attrs['na_fixed'] = day._v_attrs['na_fixed']
    return 'It took {} in total'.format(time.strftime('%H:%M:%S', time.gmtime(time.time() - tic)))

In [109]:
in_path = '../data/filled_carray.h5'
out_path = '../data/stand_carray.h5'
transfer_stand(in_path, out_path, df_stand)

'It took 00:03:39 in total'

## Resample data to obtain coarser times

### Produce new dataset 

In [33]:
def transfer_resample(in_path, out_path, target_s, orig_rows=54001, n_cols=18):
    target_rows = int((orig_rows - 1) / target_s)
    # Parameters for the output hdf5 file
    filters = tb.Filters(complib='blosc:lz4hc', complevel=9)
    dtype = tb.Float32Atom()
    data_shape = (0, n_cols)
    chunkshape=(target_s, n_cols)
    tic = time.time()
    # Open file and transfer
    with tb.open_file(in_path, mode='r') as in_file, \
         tb.open_file(out_path, mode='a', filters=filters) as out_file, \
         warnings.catch_warnings() as w:
        warnings.simplefilter('ignore', tb.NaturalNameWarning)
        # Get sensor names
        sensors = eval(in_file.get_node('/2010/03/18')._v_attrs['columns'])[1:]
        for year in in_file.get_node('/'):
            for month in in_file.get_node(year):
                for day in in_file.get_node(month):
                    # Load data for that day
                    array = day[:, :]
                    # Downsample
                    new_array = array[:-1, 1:].reshape(target_rows, target_s, len(sensors)).mean(axis=1)
                    # Add new column
                    new_array = np.c_[np.empty((target_rows, 1)), new_array]
                    # Include time since epoch in new column
                    new_array[:, 0] = array[:-1:target_s, 0]
                    # Save to new hdf5 file
                    node = out_file.create_carray(obj=new_array, createparents=True,
                                                  where=month._v_pathname, name=day._v_name)
                    # Add meta: column names and NaN fixed counts
                    node._v_attrs['columns'] = day._v_attrs['columns']
                    node._v_attrs['na_fixed'] = day._v_attrs['na_fixed']
    return 'It took {} in total'.format(time.strftime('%H:%M:%S', time.gmtime(time.time() - tic)))

In [32]:
in_path = '../data/stand_carray.h5'
out_path = '../data/stand_10s.h5'
transfer_resample(in_path, out_path, target_s=10)

'It took 00:00:15 in total'

In [7]:
in_path = '../data/filled_01s.h5'
out_path = '../data/filled_10s.h5'
transfer_resample(in_path, out_path, target_s=10)

'It took 00:00:14 in total'

In [34]:
in_path = '../data/csi_haurwitz_01s.h5'
out_path = '../data/csi_haurwitz_10s.h5'
transfer_resample(in_path, out_path, target_s=10)

'It took 00:00:11 in total'

In [40]:
in_path = '../data/stand_carray.h5'
out_path = '../data/stand_30s.h5'
transfer_resample(in_path, out_path, target_s=30)

'It took 00:00:08 in total'

In [8]:
in_path = '../data/filled_01s.h5'
out_path = '../data/filled_30s.h5'
transfer_resample(in_path, out_path, target_s=30)

'It took 00:00:08 in total'

In [36]:
in_path = '../data/csi_haurwitz_01s.h5'
out_path = '../data/csi_haurwitz_30s.h5'
transfer_resample(in_path, out_path, target_s=30)

'It took 00:00:06 in total'

In [41]:
in_path = '../data/stand_carray.h5'
out_path = '../data/stand_60s.h5'
transfer_resample(in_path, out_path, target_s=60)

'It took 00:00:07 in total'

In [9]:
in_path = '../data/filled_01s.h5'
out_path = '../data/filled_60s.h5'
transfer_resample(in_path, out_path, target_s=60)

'It took 00:00:06 in total'

In [37]:
in_path = '../data/csi_haurwitz_01s.h5'
out_path = '../data/csi_haurwitz_60s.h5'
transfer_resample(in_path, out_path, target_s=60)

'It took 00:00:05 in total'

## Clear-sky index 

In [ ]:
def compute_clear_sky(df_st, ts, model='ineichen'):
    '''
    Consider that ts has length batch_size + 1 (to account for the first ts).
    '''
    ts = pd.to_datetime(ts, unit='s').tz_localize('US/Hawaii')
    sensors = df_st.index.values
    # Compute clear-sky GHI for each sensor
    cs = pd.DataFrame(columns=sensors)
    for sensor in sensors:
        aux = Location(*df_st.loc[sensor, ['Latitude', 'Longitude']].values)
        # ineichen with climatology table by default
        cs.loc[:, sensor] = aux.get_clearsky(ts, model).loc[:, 'ghi']
    return cs

### Produce new dataset

In [4]:
def transfer_csi(in_path, out_path, model, max_csi=2.0, max_rows=54001, n_cols=18):
    # Parameters for the output hdf5 file
    filters = tb.Filters(complib='blosc:lz4hc', complevel=9)
    dtype = tb.Float32Atom()
    data_shape = (0, n_cols)
    chunkshape=(max_rows, n_cols)
    df_st = pd.read_csv('../data/other/stations.txt', '\t', index_col=0)
    tic = time.time()
    # Open file and transfer
    with tb.open_file(in_path, mode='r') as in_file, \
         tb.open_file(out_path, mode='a', filters=filters) as out_file, \
         warnings.catch_warnings() as w:
        warnings.simplefilter('ignore', tb.NaturalNameWarning)
        # Get sensor names
        sensors = eval(in_file.get_node('/2010/03/18')._v_attrs['columns'])[1:]
        for year in in_file.get_node('/'):
            for month in in_file.get_node(year):
                for day in in_file.get_node(month):
                    # Load data for that day
                    array = day[:, :]
                    # Compute clear-sky index
                    cs = compute_clear_sky(df_st, array[:, 0], model=model)
                    array[:, 1:] = clearsky_index(array[:, 1:], cs.values, max_csi)
                    # Save to new hdf5 file
                    node = out_file.create_carray(obj=array, createparents=True,
                                                  where=month._v_pathname, name=day._v_name)
                    # Add meta: column names and NaN fixed counts
                    node._v_attrs['columns'] = day._v_attrs['columns']
                    node._v_attrs['na_fixed'] = day._v_attrs['na_fixed']
    return 'It took {} in total'.format(time.strftime('%H:%M:%S', time.gmtime(time.time() - tic)))

In [38]:
models = ['ineichen', 'haurwitz', 'simplified_solis']
m_id = 2
in_path = '../data/filled_01s.h5'
out_path = '../data/csi_{}_01s.h5'.format(models[m_id])
transfer_csi(in_path, out_path, models[m_id])

/home/iprado/py-3.7/lib/python3.7/site-packages/pvlib/irradiance.py:1155: RuntimeWarning: divide by zero encountered in true_divide
  clearsky_index = ghi / clearsky_ghi
/home/iprado/py-3.7/lib/python3.7/site-packages/pvlib/irradiance.py:1155: RuntimeWarning: invalid value encountered in true_divide
  clearsky_index = ghi / clearsky_ghi


'It took 02:43:13 in total'

## Irradiance map

### Produce new dataset

In [2]:
from scipy.interpolate import griddata

def interp_map(arr, xy, X, Y):
    '''
    xy : Longitude and latitude of the available sensors.
    X : x-coordinates of the mesh grid.
    Y : y-coordinates of the mesh grid.
    arr : Values from the sensors used to interpolate.
    '''
    return griddata(xy, arr, (X, Y), method='nearest')

In [5]:
def transfer_map(in_path, out_path, map_dim=(54001, 10, 10)):
    # Parameters for the output hdf5 file
    filters = tb.Filters(complib='blosc:lz4hc', complevel=9)
    dtype = tb.Float32Atom()
    chunkshape = map_dim
    df_st = pd.read_csv('../data/other/stations.txt', '\t', index_col=0)
    # Prepare coordinates for interpolation
    xy = df_st.loc[:, ['Longitude', 'Latitude']].values
    x = xy[:, 0]
    y = xy[:, 1]
    # For a 10 by 10 map
    #xnew = np.linspace(min(x) - 0.001, max(x) + 0.001, 10)
    #ynew = np.linspace(min(y) - 0.001, max(y) + 0.001, 10)
    # For a 8 by 8 map
    xnew = np.linspace(min(x) - 0.0006, max(x) + 0.0006, 8)
    ynew = np.linspace(min(y) - 0.0006, max(y) + 0.0006, 8)
    X, Y = np.meshgrid(xnew, ynew)
    tic = time.time()
    # Open file and transfer
    with tb.open_file(in_path, mode='r') as in_file, \
         tb.open_file(out_path, mode='a', filters=filters) as out_file, \
         warnings.catch_warnings() as w:
        warnings.simplefilter('ignore', tb.NaturalNameWarning)
        # Get sensor names
        sensors = eval(in_file.get_node('/2010/03/18')._v_attrs['columns'])[1:]
        for year in in_file.get_node('/'):
            for month in in_file.get_node(year):
                for day in in_file.get_node(month):
                    # Load data for that day
                    array = day[:, :]
                    # Interpolate gaps (watch the axes order!)
                    map_arr = interp_map(array[:, 1:].T, xy, X, Y).transpose((2, 0, 1))
                    # Save to new hdf5 file
                    node = out_file.create_carray(obj=map_arr, createparents=True,
                                                  where=month._v_pathname, name=day._v_name)
                    # Add meta: column names
                    node._v_attrs['columns'] = day._v_attrs['columns']
    return 'It took {} in total'.format(time.strftime('%H:%M:%S', time.gmtime(time.time() - tic)))

In [4]:
in_path = '../data/stand_01s.h5'
out_path = '../data/stand_map10_01s.h5'
transfer_map(in_path, out_path)

'It took 00:05:43 in total'

In [10]:
in_path = '../data/stand_01s.h5'
out_path = '../data/stand_map08_01s.h5'
transfer_map(in_path, out_path, map_dim=(54001, 8, 8))

'It took 00:03:47 in total'

### Resample data to obtain coarser times

In [7]:
def transfer_resample(in_path, out_path, target_s, orig_dim=(54001, 10, 10)):
    target_t = int((orig_dim[0] - 1) / target_s)
    # Parameters for the output hdf5 file
    filters = tb.Filters(complib='blosc:lz4hc', complevel=9)
    dtype = tb.Float32Atom()
    chunkshape = (target_t, *orig_dim[1:])
    tic = time.time()
    # Open file and transfer
    with tb.open_file(in_path, mode='r') as in_file, \
         tb.open_file(out_path, mode='a', filters=filters) as out_file, \
         warnings.catch_warnings() as w:
        warnings.simplefilter('ignore', tb.NaturalNameWarning)
        for year in in_file.get_node('/'):
            for month in in_file.get_node(year):
                for day in in_file.get_node(month):
                    # Load data for that day
                    array = day[:, :]
                    # Downsample
                    new_array = array[:-1, :, :].reshape((target_t, target_s, *orig_dim[1:])).mean(axis=1)
                    # Save to new hdf5 file
                    node = out_file.create_carray(obj=new_array, createparents=True,
                                                  where=month._v_pathname, name=day._v_name)
                    # Add meta: column names
                    node._v_attrs['columns'] = day._v_attrs['columns']
    return 'It took {} in total'.format(time.strftime('%H:%M:%S', time.gmtime(time.time() - tic)))

In [8]:
in_path = '../data/stand_map10_01s.h5'
out_path = '../data/stand_map10_01m.h5'
transfer_resample(in_path, out_path, target_s=60)

'It took 00:00:38 in total'

In [11]:
in_path = '../data/stand_map08_01s.h5'
out_path = '../data/stand_map08_01m.h5'
transfer_resample(in_path, out_path, target_s=60, orig_dim=(54001, 8, 8))

'It took 00:00:22 in total'